# Factor Analysis from scratch with NumPy and Pandas
> In this post, we are going to see the basics of exploratory factor analysis, a key topic of Multivariate Analysis. We will implement a small procedure to extract the factors using Pandas and NumPy.

- toc: true 
- badges: true
- comments: true
- categories: [factor analysis, numpy, pandas, multivariate analysis]
- author: Alejandro Pérez Sanjuán
- image: images/chart-preview.png

In my job, in the most recent project I was envolved into, I had to deal with hidden variables. Searching on the web I found a paper that had to solve a similar issue, and they chose to use factor analysis.

I honestly knew nothing about factor analysis, so I started to search and read information from different sources. I found that most resources were too opaque, while others didn't explain the concepts in detail and relied its content on explaining the extraction process using libraries such [psych](https://cran.r-project.org/web/packages/psych/index.html) or programs like [SPSS](https://www.ibm.com/analytics/spss-statistics-software).

This post aims to solve these problems by explaining the theory (I should say aggregating theory from different sources listed at the end) and then implementing things from scratch. Let's begin!

## 1. Theory
### 1.1. Definitions
Factor analysis is a multivariate method that aims to represent $y_{p}$ observable variables as a linear combination of $f_{m}$ factors, with $m < p$. The factor analysis model can be expressed as follows {% fn 1 %} {% fn 2 %}:
\begin{matrix}
   y_{1} - \mu_{1} = a_{11} f_{1} + a_{12} f_{2} + ... + a_{1m} f_{m} + \epsilon_{1} \\
   y_{2} - \mu_{2} = a_{11} f_{1} + a_{12} f_{2} + ... + a_{1m} f_{m} + \epsilon_{1} \\
   \vdots \\
   y_{p} - \mu_{p} = a_{p1} f_{1} + a_{p2} f_{2} + ... + a_{pm} f_{m} + \epsilon_{p}
\end{matrix}

where $\mu$ is the mean vector, $a_{ij}$ are the loadings and $\epsilon$ is the error vector. The loadings are nothing more than regression weights for the factors. They can be viewed as the contributions of each factor in estimating the original variables.

Why don't we use a simple regression in this case? The answer is pretty obvious: we cannot observe the independent variables, which are the hidden factors.

The errors $\epsilon$ serve to indicate that the hypothesized relationships are not exact {% fn 3 %}. That is, we try to explain the variance of our variables, but we assume we cannot explain all the variance. These errors account for this lack of precision.

We can write the factor analysis model in matrix notation:
\begin{equation}
Y - \mu = A \cdot F + \varepsilon 
\end{equation}


### 1.2. Assumptions
The first assumtion is that the existance of these hidden variables is an hypothesis; that is, we cannot assume that these variables really exist.

Other assumptions are listed below {% fn 1 %} {% fn 2 %}:
* $E[F] = 0$
* $cov[F] = E[FF^{T}] = I$, remember $I$ is the identity matrix.
* $cov[\epsilon_{i}, \epsilon_{j}] = 0$, no association between errors.
* $E[\varepsilon] = 0$
* $cov[\varepsilon] = \Psi $, where

$$
\Psi = 
\left(
\begin{matrix}
\Psi_{1} & 0 & ... & 0 \\
0 & \Psi_{2} & ... & 0 \\
0 & 0 & ... & \Psi_{p} \\
\end{matrix}
\right)
$$

* $cov(F, \varepsilon) = 0$, they are independent.

### 1.3. Factor extraction.
In this post we are only going to see one type of extraction method: principal component. If you work with data, you probably heard these 2 words before. The name for this extraction procedure may be a little misleading but it is true that FA and PCA are related.

With this procedure, we try to approximate the covariance/correlation matrix $S$ (they are equal if the data ara standardized) with this expression:
$$
S \cong A A^{T} + \Psi
$$

$A$ is the loading matrix while $\Psi$ is the uniqueness matrix. The uniqueness accounts for the proportion of variance that is inherent to each variable and cannot be explain by the factors.

### 1.4. Which number of factors should I use?
A common criterion to use when deciding the number of factors is the scree plot.

## 2. Implementation
For the implementation, we will use `Numpy` and `Pandas`. NumPy will be used to handle almost all calculations while Pandas will take care of keeping the format in a nice and suited way for further processing.

In [1]:
import numpy as np
import pandas as pd

We are going to work with the following dummy data. It is a small matrix on purpose, to make it easier to follow the steps.

In [2]:
# Data
a = np.array([[1, 5, 5, 1, 1],
              [8, 9, 7, 9, 8],
              [9, 8, 9, 9, 8],
              [9, 9, 9, 9, 9],
              [1, 9, 1, 1, 9],
              [9, 7, 7, 9, 9],
              [9, 7, 9, 9, 7]])

# format
columns = ['Kind', 'Intelligent', 'Happy', 'Likeable', 'Just']
index = ['FSM1', 'Sister', 'FSM2', 'Father', 'Teacher', 'MSM', 'FSM3']

# convert to dataframe
data = pd.DataFrame(a, columns=columns, index=index)
data

,Kind,Intelligent,Happy,Likeable,Just
FSM1,1,5,5,1,1
Sister,8,9,7,9,8
FSM2,9,8,9,9,8
Father,9,9,9,9,9
Teacher,1,9,1,1,9
MSM,9,7,7,9,9
FSM3,9,7,9,9,7


To create our factor analysis procedure we will follow [Scikit-Learn](https://scikit-learn.org/stable/) philosophy, which I find very well designed. We will create a class with 2 main public methods: `fit()` and `transform()`. The rest of the methods will serve as helpers of the main functions.

In [3]:
class EFA:
    """Exploratory factor analysis.
    
    A class to perform exploratory factor analysis using the
    principal component method to extract the factors.
    
    Attributes
    ----------
    loadings : pandas.DataFrame
        The loadings matrix. Default to None if fit() has not 
        been called.
    communalities : pandas.DataFrame
        The communalities matrix. Default to None if fit() has
        not been called.
    variance_summary : pandas.DataFrame
        The variance summary of the data. Default to None if fit()
        has not been called.
    original_data : pandas.DataFrame
        The original data. Deafault to None if 'fit()' has not been
        called yet.
    
    Parameters
    ----------
    n_factors : int
        The number of factors to extract.
    is_corr : bool, optional, default: False
        If True, the passed data in 'fit()' must be a correlation matrix.
    
    """
    def __init__(self, n_factors, is_corr=False):
        self.n_factors = n_factors
        self.is_corr = is_corr
        self.loadings_ = None
        self.communalities_ = None
        self.variance_summary_ = None
    
    def fit(self, x):
        """Fits.
        
        Fits the factor analysis model and computes loadings, communalities
        and uniquenes matrices.
        
        Parameters
        ----------
        x : pandas.DataFrame
            The data to fit the model.
        """

        
        corr_m = self._validate_input(x)

        # store data
        self.original_data = corr_m
        
        # Principal factor extraction
        self._principal_factor(corr_m)
        
    def _validate_input(self, x):
        """Validates input.
        
        Checks whether the input is appropiate or not.
        
        Parameters
        ----------
        x : pandas.DataFrame
            The input to check.
        """
        if not isinstance(x, pd.DataFrame):
            raise ValueError("Please, pass 'x' as pandas.DataFrame")
        
        if self.is_corr:
            assert len(x) == len(x.columns), "Data is not symmetric"
            corr_m = x
        else:
            corr_m = x.corr()
        
        return corr_m
    
    def transform(self, x):
        """Transforms.
        
        """
        pass
    
    def _create_variance_summary(self, w):
        """Creates variances summary.
        
        Creates a DataFrame containing the variance summary of
        the given data.
        
        Parameters
        ----------
        w : numpy.array
            An array containing the eigenvalues.
        
        """
        # Round to 3 decimals
        w = np.round(w, 3)
        
        total = w.sum()
        perc_var = np.array([100 * (x/total) for x in w])
        
        aux_dict = {"Eigenvalues": w, 
                    "Variance %": perc_var, 
                    "Cum. Variance %": perc_var.cumsum()}
        
        self.variance_summary_ = pd.DataFrame(aux_dict,
                                             index=self.original_data.index)
    
    def _principal_factor(self, corr_matrix):
        """Principal factor procedure"""
        orig_idx = self.original_data.index
        
        # Singular value decomposition
        u, s, v = np.linalg.svd(corr_matrix)
        
        # compute % of each eigenvalue
        self._create_variance_summary(s)
        
        # Compute loadings and subset them
        a = pd.DataFrame(u * np.sqrt(s), index=orig_idx)
        self.loadings_ = a.iloc[:, 0:self.n_factors]
        
        # Compute communalities
        self._compute_communalities()
        
    def _compute_communalities(self):
        """Computes communalities"""
        orig_idx = self.original_data.index
        
        # communalities computation
        squared_sum = (self.loadings_ ** 2).sum(axis=1)
        
        # Format dataframe
        self.communalities_ = pd.DataFrame(round(squared_sum, 3),
                                           index=orig_idx,
                                           columns=['Communalities'])

In [4]:
data.corr()

,Kind,Intelligent,Happy,Likeable,Just
Kind,1.000000,0.295536,0.880572,0.995429,0.544567
Intelligent,0.295536,1.000000,-0.021744,0.326164,0.837288
Happy,0.880572,-0.021744,1.000000,0.866667,0.130337
Likeable,0.995429,0.326164,0.866667,1.000000,0.544016
Just,0.544567,0.837288,0.130337,0.544016,1.000000


In [10]:
fa = EFA(n_factors=2, is_corr=False)

In [12]:
fa.fit(data)

In [7]:
fa.variance_summary_

,Eigenvalues,Variance %,Cum. Variance %
Kind,3.263,65.26,65.26
Intelligent,1.538,30.76,96.02
Happy,0.168,3.36,99.38
Likeable,0.031,0.62,100.00
Just,0.000,0.00,100.00


In [8]:
fa.loadings_

,0,1
Kind,-0.969455,0.231148
Intelligent,-0.519402,-0.806945
Happy,-0.784517,0.587241
Likeable,-0.970870,0.209949
Just,-0.703964,-0.666927


In [9]:
fa.communalities_

,Communalities
Kind,0.993
Intelligent,0.921
Happy,0.960
Likeable,0.987
Just,0.940


## References
{{ 'C.M Cuadras - Nuevos Métodos de Análisis Multivariante. CMC Editions. Barcelona, 2010 ' | fndetail: 1 }}
{{ 'Albin C. Rencher - Methods of Multivariate Analysis. John Wiley & Sons. ' | fndetail: 2 }}
{{ 'Peter Tryfos - [Chapter 14: Factor Analysis](http://www.yorku.ca/ptryfos/f1400.pdf)!' | fndetail: 3 }}
{{ 'NCSS - [Chapter 420: Factor Analysis](https://ncss-wpengine.netdna-ssl.com/wp-content/themes/ncss/pdf/Procedures/NCSS/Factor_Analysis.pdf)!' | fndetail: 4 }}